In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Load dataset from CSV
def load_fashion_mnist(csv_path):
    data = pd.read_csv(csv_path).values  
    labels = data[:, 0]  
    images = data[:, 1:] 

    # Normalize pixel values to [0,1]
    images = images.astype(np.float32) / 255.0

    return images, labels

In [3]:
# Load train and test datasets
train_images, train_labels = load_fashion_mnist("datasets/fashion-mnist_train.csv")
test_images, test_labels = load_fashion_mnist("datasets/fashion-mnist_test.csv")

# Split train into (train + validation)
num_train = int(0.8 * train_images.shape[0])
val_images, val_labels = train_images[num_train:], train_labels[num_train:]
train_images, train_labels = train_images[:num_train], train_labels[:num_train]

print(f"Train: {train_images.shape}, Validation: {val_images.shape}, Test: {test_images.shape}")

Train: (48000, 784), Validation: (12000, 784), Test: (10000, 784)


In [4]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=10, pooling_type='max'):
        super(CNNModel, self).__init__()
        self.pooling_type = pooling_type

        # Input: 1x28x28 -> Output: 6x28x28
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)

        # Input: 6x28x28 -> Output: 6x14x14
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Input: 6x14x14 -> Output: 16x10x10
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)

        # Input: 16x10x10 -> Output: 16x5x5
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Input: 16x5x5 -> Output: 120
        self.fc1 = nn.Linear(400, 120)

        # Input: 120 -> Output: 84
        self.fc2 = nn.Linear(120, 84)

        # Input: 84 -> Output: num_classes
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x, extract_features=False):
        x = self.conv1(x)
        x = F.tanh(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.tanh(x)
        x = self.pool2(x)

        if extract_features:
            return x  
        
        x = self.fc1(x.view(-1, 400))
        x = F.tanh(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1) 
        return x

In [5]:
# X_train = train_images.reshape(-1, 1, 28, 28)
# X_train = torch.from_numpy(X_train).float()
# y_train = torch.from_numpy(train_labels).long()

# X_val = val_images.reshape(-1, 1, 28, 28)
# X_val = torch.from_numpy(X_val).float()
# y_val = torch.from_numpy(val_labels).long()

# train_dataset = TensorDataset(X_train, y_train)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# cnn_model = CNNModel(num_classes=10)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# num_epochs = 25
# for epoch in range(num_epochs):
#     cnn_model.train()
#     total_loss = 0
#     for images, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = cnn_model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# print("Training complete!")


In [6]:
X_train = train_images.reshape(-1, 1, 28, 28)
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(train_labels).long()

X_val = val_images.reshape(-1, 1, 28, 28)
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(val_labels).long()

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

cnn_model = CNNModel(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

num_epochs = 30
for epoch in range(num_epochs):
    cnn_model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    # Training Loop
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        # Compute training accuracy
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = (correct_train / total_train) * 100

    # Validation Loop
    cnn_model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = cnn_model(images)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_accuracy = (correct_val / total_val) * 100

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%")

print("Training complete!")

Epoch 1, Loss: 2868.6979, Train Acc: 63.89%, Val Acc: 74.15%
Epoch 2, Loss: 2780.5287, Train Acc: 74.75%, Val Acc: 75.44%
Epoch 3, Loss: 2773.8100, Train Acc: 75.50%, Val Acc: 75.68%
Epoch 4, Loss: 2770.7943, Train Acc: 75.81%, Val Acc: 75.86%
Epoch 5, Loss: 2768.0882, Train Acc: 76.17%, Val Acc: 76.14%
Epoch 6, Loss: 2732.5072, Train Acc: 81.12%, Val Acc: 82.09%
Epoch 7, Loss: 2715.7484, Train Acc: 83.24%, Val Acc: 82.99%
Epoch 8, Loss: 2712.0817, Train Acc: 83.68%, Val Acc: 83.43%
Epoch 9, Loss: 2709.0167, Train Acc: 84.14%, Val Acc: 83.61%
Epoch 10, Loss: 2707.0726, Train Acc: 84.35%, Val Acc: 83.88%
Epoch 11, Loss: 2705.5796, Train Acc: 84.54%, Val Acc: 84.17%
Epoch 12, Loss: 2692.7053, Train Acc: 86.45%, Val Acc: 86.88%
Epoch 13, Loss: 2677.0939, Train Acc: 88.52%, Val Acc: 87.64%
Epoch 14, Loss: 2671.1116, Train Acc: 89.36%, Val Acc: 88.47%
Epoch 15, Loss: 2665.8937, Train Acc: 90.09%, Val Acc: 89.17%
Epoch 16, Loss: 2662.0913, Train Acc: 90.54%, Val Acc: 89.08%
Epoch 17, Loss: 2

In [7]:
# Evaluate model from the CNN itself
X_test = test_images.reshape(-1, 1, 28, 28)
X_test = torch.from_numpy(X_test).float()
Y_test = torch.from_numpy(test_labels).long()

cnn_model.eval()
outputs = cnn_model(X_test)
_, predictions = torch.max(outputs, 1)
accuracy = torch.sum(predictions == Y_test).item() / len(Y_test) * 100
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 90.06%
